# First draft scrips for parsing through Mexican Congress data

## Importing necessary libraries

In [3]:
# importing required modules
from PyPDF2 import PdfReader
from pdf2image import convert_from_path
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/5.3.1/bin/tesseract'
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import requests
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
import numpy as np
from lxml import etree
import re
from selenium import webdriver
import time

## Federal Senate Database

In [9]:
def get_senators():
    senators_url = 'https://www.senado.gob.mx/65/datosAbiertos/senadoresDatosAb.json'
    senators_json = requests.get(senators_url).json()
    senators = pd.DataFrame.from_dict(senators_json)
    senators = senators.rename(columns={"idSenador": "senator_id"})
    return senators

In [10]:
senators = get_senators()

In [9]:
def get_commissions():
    commissions_url = 'https://www.senado.gob.mx/65/datosAbiertos/comisiones.json'
    commissions_json = requests.get(commissions_url).json()
    commissions = pd.DataFrame.from_dict(commissions_json)
    return commissions

In [4]:
def get_initiatives():
    init_64_url = 'https://www.senado.gob.mx/65/datosAbiertos/iniciativa_64.json'
    init_65_url = 'https://www.senado.gob.mx/65/datosAbiertos/iniciativa_65.json'
    
    init_64_json = requests.get(init_64_url).json()
    init_65_json = requests.get(init_65_url).json()
    
    init_64 = pd.DataFrame.from_dict(init_64_json)
    init_65 = pd.DataFrame.from_dict(init_65_json)
    
    initiatives = pd.concat([init_64, init_65])
    
    initiatives['fecha_presentacion'] = pd.to_datetime(initiatives['fecha_presentacion'],errors='coerce')
    initiatives['fecha_aprobacion'] = pd.to_datetime(initiatives['fecha_aprobacion'],errors='coerce')
    
    initiatives = initiatives.set_index('id')
        
    return initiatives

In [5]:
initiatives = get_initiatives()

In [7]:
initiatives

,titulo,fecha_presentacion,sentido_dictamen,url_gaceta,estado,sintesis,legislatura,senadores,comisiones,leyes_modifica,tipo,camara_origen,fecha_aprobacion
id,,,,,,,,,,,,,
724928,Proyecto de decreto por el que se adiciona una...,2021-08-25,No Aplica,120002,Pendiente,Propone crear los certificados de vacunación e...,64,Geovanna del Carmen Bañuelos de La Torre (PT)<br>,Salud (Coordinadora) <br>Estudios Legislativos...,Ley General de Salud<br>,iniciativa,No Aplica,NaT
724929,Proyecto de decreto por el que se reforman las...,2021-08-25,No Aplica,119909,Pendiente,Propone establecer como materia de salubridad ...,64,Martha Cecilia Márquez Alvarado (PAN)<br>,Salud (Coordinadora) <br>Estudios Legislativos...,Ley de los Institutos Nacionales de Salud<br>L...,iniciativa,No Aplica,NaT
724930,Proyecto de decreto por el que se adicionan lo...,2021-08-25,No Aplica,120038,Pendiente,Propone establecer que en caso excepcional de ...,64,Ricardo Monreal Ávila (MORENA)<br>,Justicia (Coordinadora) <br>Estudios Legislati...,Código Civil Federal<br>,iniciativa,No Aplica,NaT
724922,Proyecto de decreto por el que se reforma el a...,2021-08-18,No Aplica,119728,Pendiente,Propone aumentar las penas y sanciones para qu...,64,Alejandro Armenta Mier (MORENA)<br>,Justicia (Coordinadora) <br>Estudios Legislati...,Código Penal Federal<br>,iniciativa,No Aplica,NaT
724923,Proyecto de decreto por el que se reforma el p...,2021-08-18,No Aplica,119734,Pendiente,"Propone que la administración, vigilancia y di...",64,Jesús Lucía Trasviña Waldenrath (MORENA)<br>,Puntos Constitucionales (Coordinadora) <br>Est...,Constitución Política de los Estados Unidos Me...,iniciativa,No Aplica,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
731382,Proyecto de decreto por el que se reforma la L...,2021-09-02,No Aplica,120214,Pendiente,Propone establecer que los contenidos de plane...,65,Lilia Margarita Valdez Martínez (MORENA)<br>,Educación (Coordinadora) <br>Estudios Legislat...,Ley General de Educación<br>,iniciativa,No Aplica,NaT
731383,Proyecto de decreto por el que se adiciona un ...,2021-09-02,No Aplica,120215,Pendiente,Propone que los enfermeros de nivel técnico pr...,65,Arturo Bours Griffith (MORENA)<br>,Salud (Coordinadora) <br>Estudios Legislativos...,Ley General de Salud<br>,iniciativa,No Aplica,NaT
731384,Proyecto de decreto por el que se reforman div...,2021-09-02,No Aplica,120216,Pendiente,Propone que se considere la responsabilidad de...,65,Ricardo Monreal Ávila (MORENA)<br>,Comunicaciones y Transportes (Coordinadora) <b...,Ley de Aviación Civil<br>,iniciativa,No Aplica,NaT


In [8]:
initiatives["sintesis"][6]

'Propone incluir el derecho a una educación alimentaria nutricional para prevenir y concientizar a la sociedad sobre las enfermedades crónicas causadas por una mala y deficiente alimentación.'

In [35]:
words = []
for initiative in initiatives["sintesis"]: 
    print(initiative)
    words.append(len(initiative))
sum(words)

Propone crear los certificados de vacunación extraordinaria o por acción extraordinaria en materia de Salubridad General físicos o digitales.

Estos se expedirán cuando las personas hayan obtenido las vacunas necesarias por la autoridad competente; en el caso de pandemia internacional, la Secretaría de Salud (SS), deberá coordinarse con la Secretaría de Relaciones Exteriores y la Secretaría de Gobernación para coordinar políticas de ingreso al país.

Además, crea el Sistema Informático Nacional para la verificación y validez de estos certificados para los tramites que así lo requieran.

Faculta a la SS para publicar un reglamento en el que se determinan los mecanismos, sistemas de seguridad, funcionamiento y obligaciones relacionadas.
Propone establecer como materia de salubridad general la atención médica, preferentemente en beneficio de grupos vulnerables y la Protección Social la cual estará a cargo de la Secretaría de Salud.

Crea el Sistema de Protección Social en Salud, que consi

2289995

In [36]:
proposals = get_proposals()
proposals

,id,titulo,fecha_presentacion,sentido_dictamen,url_gaceta,estado,sintesis,legislatura,senadores,comisiones,leyes_modifica,tipo,camara_origen,fecha_aprobacion
0,727558,Punto de acuerdo por el que se exhorta al titu...,2021-08-25,No Aplica,117641,Desechada,PRIMERO. La Comisión Permanente del H. Congres...,64,Senadores integrantes del PAN (PAN)<br>,Gobernación (Coordinadora) <br>,No Alpica,proposicion,No Aplica,2022-11-04
1,727606,Punto de acuerdo por el que se exhorta al Ejec...,2021-08-25,No Aplica,117951,Aprobada,PRIMERO.- El Senado de la República exhorta re...,64,Damián Zepeda Vidales (PAN)<br>,Seguridad Pública (Coordinadora) <br>,No Alpica,proposicion,No Aplica,2021-12-07
2,727655,Punto de acuerdo por el que se exhorta al titu...,2021-08-25,No Aplica,118576,Desechada,ÚNICO.- La Comisión Permanente del Congreso de...,64,Senadores integrantes del PAN (PAN)<br>,Gobernación (Coordinadora) <br>,No Alpica,proposicion,No Aplica,2022-11-24
3,727657,Punto de acuerdo por el que la Comisión Perman...,2021-08-25,No Aplica,118580,Aprobada,ÚNICO.- La Comisión Permanente del Congreso de...,64,Senadores integrantes del PAN (PAN)<br>,Seguridad Pública (Coordinadora) <br>,No Alpica,proposicion,No Aplica,2021-12-07
4,727665,Punto de acuerdo por el que se exhorta al Pres...,2021-08-25,No Aplica,118750,Aprobada,ÚNICO.- La Comisión Permanente del H. Congreso...,64,Senadores integrantes del PAN (PAN)<br>,Seguridad Pública (Coordinadora) <br>,No Alpica,proposicion,No Aplica,2021-12-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,733600,Punto de acuerdo por el que se exhorta al titu...,2021-09-02,No Aplica,120141,Desechada,PRIMERO.- El Senado de la República exhorta re...,65,Mario Zamora Gastélum (PRI)<br>,Gobernación (Coordinadora) <br>,No Alpica,proposicion,No Aplica,2022-11-04
1309,733601,Punto de acuerdo por el que se exhorta a la Se...,2021-09-02,No Aplica,120099,Aprobada,ÚNICO. Se exhorta al titular del Poder Ejecuti...,65,José Clemente Castañeda Hoeflich (MC)<br>Verón...,Urgente Resolución (Coordinadora) <br>,No Alpica,proposicion,No Aplica,2021-09-02
1310,733602,Punto de acuerdo por el que se exhorta a la Gu...,2021-09-02,No Aplica,120105,Aprobada,ÚNICO. El Senado de la República exhorta respe...,65,Lilly Téllez García (PAN)<br>,Seguridad Pública (Coordinadora) <br>,No Alpica,proposicion,No Aplica,2021-12-07
1311,733603,Punto de acuerdo por el que se exhorta a la Co...,2021-09-02,No Aplica,120228,Aprobada,PRIMERO. El Senado de la República exhorta res...,65,Raúl Paz Alonzo (PAN)<br>,Recursos Hidráulicos (Coordinadora) <br>,No Alpica,proposicion,No Aplica,2021-12-14


In [37]:
words = []
for proposal in proposals["sintesis"]: 
    words.append(len(proposal))
sum(words)

3147718

In [95]:
print(type(initiatives["sintesis"][0]))

for titulo in initiatives["sintesis"][0]:
    print(titulo)
    print("-"*100)

<class 'pandas.core.series.Series'>
Propone crear los certificados de vacunación extraordinaria o por acción extraordinaria en materia de Salubridad General físicos o digitales.

Estos se expedirán cuando las personas hayan obtenido las vacunas necesarias por la autoridad competente; en el caso de pandemia internacional, la Secretaría de Salud (SS), deberá coordinarse con la Secretaría de Relaciones Exteriores y la Secretaría de Gobernación para coordinar políticas de ingreso al país.

Además, crea el Sistema Informático Nacional para la verificación y validez de estos certificados para los tramites que así lo requieran.

Faculta a la SS para publicar un reglamento en el que se determinan los mecanismos, sistemas de seguridad, funcionamiento y obligaciones relacionadas.
----------------------------------------------------------------------------------------------------
Propone establecer que el Estado garantice el acceso y continuidad de la atención de salud mental y adicciones con car

In [12]:
initiatives["senadores"] = initiatives["senadores"].apply(lambda x: x.split("<br>"))
initiatives["senadores"][50]

50       [José Alejandro Peña Villa (MORENA), ]
50    [Nuvia Magdalena Mayorga Delgado (PRI), ]
Name: senadores, dtype: object

In [7]:
def get_proposals():
    prop_64_url = 'https://www.senado.gob.mx/65/datosAbiertos/proposicion_64.json'
    prop_65_url = 'https://www.senado.gob.mx/65/datosAbiertos/proposicion_65.json'
    
    prop_64_json = requests.get(prop_64_url).json()
    prop_65_json = requests.get(prop_65_url).json()
    
    prop_64 = pd.DataFrame.from_dict(prop_64_json)
    prop_65 = pd.DataFrame.from_dict(prop_65_json)
    
    proposals = pd.concat([prop_64, prop_65])
    
    proposals['fecha_presentacion'] = pd.to_datetime(proposals['fecha_presentacion'],errors='coerce')
    proposals['fecha_aprobacion'] = pd.to_datetime(proposals['fecha_aprobacion'],errors='coerce')
    
    return proposals

In [8]:
proposals = get_proposals()

In [12]:
def get_senator_attendance():
    
    senators = get_senators()
    
    senator_ids = senators["senator_id"].tolist()
    
    senator_attendance = pd.DataFrame()
    senator_attendance["senator_id"] = ""
    senator_attendance["session_date"] = ""
    senator_attendance["attendance_record"] = ""

    counter = 0
    for sen in senator_ids:
        url = f'https://www.senado.gob.mx/65/asistencias/{sen}#info'
        html = requests.get(url)
        content = BeautifulSoup(html.text, 'html.parser')
        content_x = etree.HTML(str(content))
        dates = content_x.xpath('//*[@id="imPage"]/div[7]/div[2]/div/div[2]/section/div/div/table/tbody//a')
        att_records = content_x.xpath('//*[@id="imPage"]/div[7]/div[2]/div/div[2]/section/div/div/table/tbody//strong')
        for i in range(len(dates)):
            senator_attendance.at[i+counter, 'senator_id'] = sen
            senator_attendance.at[i+counter, 'session_date'] = dates[i].text
            senator_attendance.at[i+counter, 'attendance_record'] = att_records[i].text
        counter += len(dates)

    senator_attendance["attendance_score"] = senator_attendance["attendance_record"].copy()
    senator_attendance["attendance_score"] = senator_attendance["attendance_score"].map(lambda x: 1 if x == "Asistencia" else 0)
    senator_attendance = pd.merge(senator_attendance, senators[['senator_id','Fraccion', 'Estado', 'Apellidos', 'Nombre', 'tipoEleccion']], on='senator_id', how='left')

    senator_attendance["full_name"] = senator_attendance['Nombre'] + " " + senator_attendance['Apellidos']
    
    senator_attendance = senator_attendance.groupby(['senator_id', 'full_name', 'Fraccion', 'Estado', 'tipoEleccion'], as_index=False)[['attendance_score']].mean()

    return senator_attendance

In [84]:
initiatives = get_initiatives()
initiatives["sintesis"] = initiatives["sintesis"].astype(str)

In [85]:
initiatives.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5520 entries, 0 to 1871
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  5520 non-null   object        
 1   titulo              5520 non-null   object        
 2   fecha_presentacion  5520 non-null   datetime64[ns]
 3   sentido_dictamen    5520 non-null   object        
 4   url_gaceta          5520 non-null   object        
 5   estado              5520 non-null   object        
 6   sintesis            5520 non-null   object        
 7   legislatura         5520 non-null   object        
 8   senadores           5520 non-null   object        
 9   comisiones          5520 non-null   object        
 10  leyes_modifica      5520 non-null   object        
 11  tipo                5520 non-null   object        
 12  camara_origen       5520 non-null   object        
 13  fecha_aprobacion    1136 non-null   datetime64[n

In [13]:
senator_attendance = get_senator_attendance()

In [14]:
senator_attendance.sort_values(by="attendance_score").head(20)

,senator_id,full_name,Fraccion,Estado,tipoEleccion,attendance_score
102,1252,Carlos Humberto Aceves del Olmo,PRI,Lista Nacional,Senador Electo por Representación Proporcional...,0.575658
6,1048,Jaime Bonilla Valdez,PT,Baja California,Senador Electo por el Principio de Mayoría Rel...,0.642857
24,1083,Eva Eugenia Galaz Caletti,morena,Coahuila de Zaragoza,Senadora Electa por el Principio de Mayoría Re...,0.688963
117,1279,Ifigenia Martínez Hernández,morena,Lista Nacional,Senadora Electa por Representación Proporciona...,0.740132
10,1056,Jesús Lucía Trasviña Waldenrath,morena,Baja California Sur,Senadora Electa por el Principio de Mayoría Re...,0.756579
81,1201,Ismael García Cabeza de Vaca,PAN,Tamaulipas,Senador Electo por el Principio de Primera Min...,0.792763
3,1042,Martha Cecilia Márquez Alvarado,PT,Aguascalientes,Senadora Electa por el Principio de Mayoría Re...,0.799308
105,1258,Beatriz Elena Paredes Rangel,PRI,Lista Nacional,Senadora Electa por Representación Proporciona...,0.802632
67,1171,Mayuli Latifa Martínez Simón,PAN,Quintana Roo,Senadora Electa por el Principio de Primera Mi...,0.804054
97,1242,Indira de Jesús Rosales San Román,PAN,Lista Nacional,Senadora Electa por Representación Proporciona...,0.811644


In [15]:
def get_senator_voting():
    """Returns dataframe with senator voting history for current senators."""
    
    #Create list of senators from senator dataframe.
    senator_ids = get_senators()["senator_id"].tolist()

    #Create dataframe to store voting data.
    senator_voting = pd.DataFrame()
    senator_voting["senator_id"] = ""
    senator_voting["voting_date"] = ""
    senator_voting["vote_id"] = ""
    senator_voting["vote_description"] = ""
    senator_voting["vote_position"] = ""

    #Script that scrapes senate website for voting data. First, all except voting date.
    sens = len(senator_ids)
    for sen in senator_ids:
        print(f"Beginning to process all votes for senator # {sen}. {sens} left to process.")
        url = f'https://www.senado.gob.mx/65/votaciones/{sen}#info'
        html = requests.get(url)
        content = BeautifulSoup(html.text, 'html.parser')
        vote_positions = content.find_all(class_ = "col-sm-1 text-center")
        vote_descriptions = content.find_all(class_ = "col-sm-11")
        votes = len(vote_descriptions)
        for i in range(len(vote_positions)):
            vote_id = vote_descriptions[i].findChildren()[0]['href'].split("/")[3]
#             start_time = time.time()
            senator_voting.at[i, 'senator_id'] = sen
            senator_voting.at[i, 'vote_position'] = vote_positions[i].text.strip()
            senator_voting.at[i, 'vote_description'] = vote_descriptions[i].text.strip()
            senator_voting.at[i, 'vote_id'] = vote_id
            votes -= 1
#             end_time = time.time()
#             time_spent = round(end_time - start_time)
#             time_left = votes * time_spent
#             print(f"{round(time_left/60)} min left.")
        sens -= 1
    
    #Script that scrapes Senate website for voting data. Secondly, voting dates.
    vote_ids = senator_voting["vote_id"].tolist()
    votes = len(vote_ids)
    for i in range(len(vote_ids)):
        print(f"Working on vote {senator_voting.at[i, 'vote_id']}. {votes} votes left.")
        start_time = time.time()
        vote_date_url = f"https://www.senado.gob.mx/65/votacion/{senator_voting.at[i, 'vote_id']}"
        html = requests.get(vote_date_url)
        content = BeautifulSoup(html.text, 'html.parser')
        vote_date = content.find_all(class_ = "col-sm-12 text-center")[1].text
        senator_voting.at[i, 'voting_date'] = vote_date
        votes -= 1
        end_time = time.time()
        time_spent = round(end_time - start_time)
        time_left = votes * time_spent
        print(f"{round(time_left/60)} min left.")
    
    
    return senator_voting

In [16]:
senator_voting = get_senator_voting()

Beginning to process all votes for senator # 1160. 128 left to process.
Beginning to process all votes for senator # 1162. 127 left to process.
Beginning to process all votes for senator # 1247. 126 left to process.
Beginning to process all votes for senator # 1101. 125 left to process.
Beginning to process all votes for senator # 1052. 124 left to process.
Beginning to process all votes for senator # 1147. 123 left to process.
Beginning to process all votes for senator # 1262. 122 left to process.
Beginning to process all votes for senator # 1201. 121 left to process.
Beginning to process all votes for senator # 1207. 120 left to process.
Beginning to process all votes for senator # 1246. 119 left to process.
Beginning to process all votes for senator # 1044. 118 left to process.
Beginning to process all votes for senator # 1171. 117 left to process.
Beginning to process all votes for senator # 1159. 116 left to process.
Beginning to process all votes for senator # 1213. 115 left to p

Beginning to process all votes for senator # 1205. 13 left to process.
Beginning to process all votes for senator # 1107. 12 left to process.
Beginning to process all votes for senator # 1109. 11 left to process.
Beginning to process all votes for senator # 1276. 10 left to process.
Beginning to process all votes for senator # 1062. 9 left to process.
Beginning to process all votes for senator # 1056. 8 left to process.
Beginning to process all votes for senator # 1097. 7 left to process.
Beginning to process all votes for senator # 1091. 6 left to process.
Beginning to process all votes for senator # 1274. 5 left to process.
Beginning to process all votes for senator # 1273. 4 left to process.
Beginning to process all votes for senator # 1169. 3 left to process.
Beginning to process all votes for senator # 1406. 2 left to process.
Beginning to process all votes for senator # 1458. 1 left to process.
Working on vote 4385. 1170 votes left.
39 min left.
Working on vote 4366. 1169 votes l

34 min left.
Working on vote 4244. 1029 votes left.
34 min left.
Working on vote 4241. 1028 votes left.
34 min left.
Working on vote 4242. 1027 votes left.
34 min left.
Working on vote 4243. 1026 votes left.
34 min left.
Working on vote 4237. 1025 votes left.
34 min left.
Working on vote 4238. 1024 votes left.
34 min left.
Working on vote 4239. 1023 votes left.
34 min left.
Working on vote 4240. 1022 votes left.
34 min left.
Working on vote 4226. 1021 votes left.
34 min left.
Working on vote 4227. 1020 votes left.
34 min left.
Working on vote 4228. 1019 votes left.
34 min left.
Working on vote 4229. 1018 votes left.
34 min left.
Working on vote 4230. 1017 votes left.
34 min left.
Working on vote 4231. 1016 votes left.
34 min left.
Working on vote 4232. 1015 votes left.
34 min left.
Working on vote 4233. 1014 votes left.
34 min left.
Working on vote 4234. 1013 votes left.
34 min left.
Working on vote 4235. 1012 votes left.
34 min left.
Working on vote 4236. 1011 votes left.
34 min left.

KeyboardInterrupt: 

In [4]:
senator_voting

NameError: name 'senator_voting' is not defined

## For PDFs that have text in them.

In [9]:
def text_from_normal_pdf(pdf_path):
    
    # creating a pdf reader object
    reader = PdfReader(pdf_path)

    # printing number of pages in pdf file
    print(len(reader.pages))

    # getting a specific page from the pdf file
    page = reader.pages[2]

    # extracting text from page
    text = page.extract_text()
    print(text)

3



## For PDFs that are images or scans of documents

In [117]:
def text_from_flat_pdf(pdf_path):
    pdfs = pdf_path
    pages = convert_from_path(pdfs, 350)
    
    i = 1
    images = []
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"  
        page.save(image_name, "JPEG")
        i = i+1
        images.append(f'/Users/jmlunamugica/sivico/code/{image_name}')
    print(images)
        
    full_text = ''
    i = 1
    for image in images:
        image = cv2.imread(image)
        # convert the image to black and white for better OCR
        ret,thresh1 = cv2.threshold(image,120,255,cv2.THRESH_BINARY)
        # pytesseract image to string to get results
        image_text = str(pytesseract.image_to_string(thresh1, config='--psm 6'))
        full_text = full_text + "page " + str(i) + ": " + image_text
        i += 1
        
    print(full_text)
    return full_text

In [118]:
text_from_flat_pdf('/Users/jmlunamugica/sivico/code/Asistencia_20230413.pdf')

['/Users/jmlunamugica/sivico/code/Page_1.jpg', '/Users/jmlunamugica/sivico/code/Page_2.jpg', '/Users/jmlunamugica/sivico/code/Page_3.jpg']
page 1: OB Go CN WER ET OR a ES
*. Ce % ee
‘ oes %,' LXV Legislatura’.  ANOI _..... Seglindo Periodo Ordinario
hy SERB OS POE ee
Cone RS Ce ee
ELA RY SE SORES RUS ER TEESE gad Bs STE ESE UES PRE SG STS Ree oar ter ck Gune rene na er eg
! REGISTRO DE ASISTENCIA DE LA SESION ORDINARIA
. CELEBRADA EL DIA JUEVES 13 DE ABRIL DE 2023.
De conformidad con los articulos 63 y 64 del reglamento del Senado de la
Republica, se presenta la siguiente relacion:
SENADORAS Y SENADORES ASISTENTES, CONFORME A LO DISPUESTO EN EL ARTICULO
58 DEL REGLAMENTO POR SISTEMA ELECTRONICO:
ABREU ARTINANO ROCIO ADRIANA
ACEVES DEL OLMO CARLOS HUMBERTO
ALVAREZ ICAZA LONGORIA EMILIO
ALVAREZ LIMA JOSE ANTONIO CRUZ
ANAYA MOTA CLAUDIA EDITH
ANORVE BANOS MANUEL
ARIAS SOLIS CRISTOBAL
ARMENTA MIER ALEJANDRO
AVILA VAZQUEZ KATYA ELIZABETH
AVILA VILLEGAS ERUVIEL
BALDERAS ESPINOZA CLAUDIA ESTH

"page 1: OB Go CN WER ET OR a ES\n*. Ce % ee\n‘ oes %,' LXV Legislatura’.  ANOI _..... Seglindo Periodo Ordinario\nhy SERB OS POE ee\nCone RS Ce ee\nELA RY SE SORES RUS ER TEESE gad Bs STE ESE UES PRE SG STS Ree oar ter ck Gune rene na er eg\n! REGISTRO DE ASISTENCIA DE LA SESION ORDINARIA\n. CELEBRADA EL DIA JUEVES 13 DE ABRIL DE 2023.\nDe conformidad con los articulos 63 y 64 del reglamento del Senado de la\nRepublica, se presenta la siguiente relacion:\nSENADORAS Y SENADORES ASISTENTES, CONFORME A LO DISPUESTO EN EL ARTICULO\n58 DEL REGLAMENTO POR SISTEMA ELECTRONICO:\nABREU ARTINANO ROCIO ADRIANA\nACEVES DEL OLMO CARLOS HUMBERTO\nALVAREZ ICAZA LONGORIA EMILIO\nALVAREZ LIMA JOSE ANTONIO CRUZ\nANAYA MOTA CLAUDIA EDITH\nANORVE BANOS MANUEL\nARIAS SOLIS CRISTOBAL\nARMENTA MIER ALEJANDRO\nAVILA VAZQUEZ KATYA ELIZABETH\nAVILA VILLEGAS ERUVIEL\nBALDERAS ESPINOZA CLAUDIA ESTHER\nBANUELOS GEOVANNA\nBENAVIDES COBOS GABRIELA\nBERMUDEZ MENDEZ JOSE ERANDI\nBOLANOS CACHO CUE RAUL\nBOTELLO MONTES

### Select region of images from which to extract text.

In [68]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt

def mark_region(image_path):
    
    im = cv2.imread(image_path)

    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9,9), 0)
    thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)
    
    # Dilate to combine adjacent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    dilate = cv2.dilate(thresh, kernel, iterations=4)
    
    # Find contours, highlight text areas, and extract ROIs
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    line_items_coordinates = []
    
    for c in cnts:
        area = cv2.contourArea(c)
        x,y,w,h = cv2.boundingRect(c)

        if y >= 600 and x <= 1000:
            if area > 10000:
                image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
                line_items_coordinates.append([(x,y), (2200, y+h)])
                return image, line_items_coordinates

        if y >= 2400 and x<= 2000:
            image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
            line_items_coordinates.append([(x,y), (2200, y+h)])
            return image, line_items_coordinates

    plt.imshow(im)

## Mexican Laws

## NLP Portion

In [3]:
import spacy

In [5]:
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

Noun phrases: ['Sebastian Thrun', 'self-driving cars', 'Google', 'few people', 'the company', 'him', 'I', 'you', 'very senior CEOs', 'major American car companies', 'my hand', 'I', 'Thrun', 'an interview', 'Recode']
Verbs: ['start', 'work', 'drive', 'take', 'tell', 'shake', 'turn', 'talk', 'say']
Sebastian Thrun PERSON
Google ORG
2007 DATE
American NORP
Thrun PERSON
Recode ORG
earlier this week DATE
